The drift term of the price process would be:

$$\mu_{S^*} = r_d - q_d = r_d - q$$

S* --> fx asset dominated in domestic ccy

$$
\sigma_{S^*}^2 = \sigma_{S*F}^2 = \sigma_{S} ^ 2 + \sigma_{F} ^ 2 + 2 * \rho * \sigma_{S} * \sigma_{F}
$$

So the price should be like: 
$$
c(S^*, T, K_d) = max(S^* - K_d, 0)
$$

$$
c(S^*, T, K_d) = e^{-r_d  \tau} * N(d_2)
$$
where: 
$$
d_1 = \frac{\log{\frac{S^*}{X_d}} + (\mu^d_{S^*} + \frac{\sigma_{S^*}}{2})  \tau}{\sigma_{S^*} \sqrt{\tau}}
$$

$$
d_2 = d_1 - \sigma_{S^*}  \sqrt{\tau}
$$

The risk-neutral probability is adjusted w.r.t the correlation risk, and since no fx denominated asset is involved in the price process, so rf is not involved in the analytical solution

In [1]:
import math
import numpy as np
from BlackScholes import BlackScholes
from digital_quanto_pricer import *
from Simulator import Simulator

In [2]:
spot = 106.6
strike = 28   # demoninated in the domestic currency
tau = 0.3
q = 0.
rd = 0.04
rf = 0.05
sigma_s = 0.4
sigma_f = 0.1
rho = 0.2
quanto_factor = 1/4

In [3]:
mu = rd - q
sigma = math.sqrt(sigma_s ** 2 + sigma_f ** 2 + 2 * rho * sigma_s * sigma_f)
s_star = spot * quanto_factor

### Analytical Solution of the Dom-Strike digital quanto call

In [4]:
call = FloatingDomStrikeDigitalQuantoCall(rd, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, quanto_factor)

In [5]:
call.price()

0.38641634543503556

### Monte Carlo Process

In [6]:
time_steps = 1500
dt = tau / time_steps
num_paths = 300000

In [7]:
%%time
payoffs = Simulator.simulate_sobol(s_star, mu, q, sigma, dt, num_paths, time_steps)

CPU times: user 44.5 s, sys: 57.5 s, total: 1min 42s
Wall time: 2min 18s


In [8]:
x = [1 if payoff[-1]>strike else 0 for payoff in payoffs]
np.mean(x) * math.exp(-rd * tau)

0.3863722690251531

### Validation for digital put

In [9]:
put = FloatingDomStrikeQuantoDigitalPut(rd, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, quanto_factor)
put.price()

0.6016553674268948

In [ ]:
math.exp(-rd * tau) * (1 - np.mean(x))

0.6016994438367774